In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"seokyunkang","key":"4a58dfcf1617eb52ddfbab75ff11ccd0"}'}

In [ ]:
ls -1ha kaggle.json

kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning이 발생하지 않도록 해줍니다.
!chmod 600 ~/.kaggle/kaggle.json
# 내가 참가한 대회 리스트 확인
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes         91           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes         98           False  
store-sales-time-series-forecasting            2030-06-30 23:59:00  Getting Started  Knowledge        936           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        161           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       1785           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      14406           False  
house-pr

In [ ]:
!kaggle competitions download -c titanic

  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.77MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 18.7MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 30.0MB/s]


In [ ]:
# Import Module
import pandas as pd
import numpy as np
import os
from tensorflow.keras.layers import Dense
from tensorflow import keras

In [ ]:
# 모든 Data를 DataFrame의 형태로 dictionary에 넣어가지고 온다.
def import_Data(file_path):
  result = dict()
  for file in os.listdir(file_path):

    file_name = file[:-4]
    result[file_name] = pd.read_csv(file)
    
  return result

In [ ]:
# 해당 경로에 있는 모든 파일을 DataFrame으로 가지고 왔다.
file_path = "/content/Dataset/"
Rawdata_dict = import_Data(file_path)

In [ ]:
dict_key = list(Rawdata_dict.keys())
dict_key

['train', 'test', 'gender_submission']

In [ ]:
Rawdata_dict[dict_key[0]]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
# 흩어져 있는 데이터를 모아 하나의 Rawdata로 만든다.
def make_Rawdata(dict_data):

  dict_key = list(dict_data.keys())
  test_Dataset = pd.merge(dict_data["gender_submission"], dict_data["test"], how = 'outer', on = "PassengerId")
  Rawdata = pd.concat([dict_data["train"], test_Dataset])
  Rawdata.reset_index(drop=True, inplace=True)

  return Rawdata

In [ ]:
Rawdata = make_Rawdata(Rawdata_dict)
Rawdata

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,0,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1305,1306,1,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1306,1307,0,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1307,1308,0,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [ ]:
from copy import copy

def remove_columns(DF, remove_list):

  # 원본 정보 유지를 위해 copy하여, 원본 Data와의 종속성을 끊었다.
  result = copy(Rawdata)

  # PassengerId를 Index로 대치
  result.set_index("PassengerId", inplace = True)

  # 불필요한 column 제거
  for column in remove_list:

    del(result[column])

  return result

In [ ]:
remove_list = ["Name", "Ticket"]
DF_Hand1 = remove_columns(Rawdata, remove_list)
DF_Hand1

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
PassengerId,,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,NaN,S
2,1,1,female,38.0,1,0,71.2833,C85,C
3,1,3,female,26.0,0,0,7.9250,NaN,S
4,1,1,female,35.0,1,0,53.1000,C123,S
5,0,3,male,35.0,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...
1305,0,3,male,NaN,0,0,8.0500,NaN,S
1306,1,1,female,39.0,0,0,108.9000,C105,C
1307,0,3,male,38.5,0,0,7.2500,NaN,S


In [ ]:
DF_Hand1.isnull().sum()

Survived       0
Pclass         0
Sex            0
Age          263
SibSp          0
Parch          0
Fare           1
Cabin       1014
Embarked       2
dtype: int64

In [ ]:
def missing_value(DF):
  # Cabin 변수를 제거
  del(DF["Cabin"])

  # 결측치가 있는 모든 행은 제거
  DF.dropna(inplace = True)

In [ ]:
#결측치 처리
missing_value(DF_Hand1)
DF_Hand1

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,S
2,1,1,female,38.0,1,0,71.2833,C
3,1,3,female,26.0,0,0,7.9250,S
4,1,1,female,35.0,1,0,53.1000,S
5,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
1301,1,3,female,3.0,1,1,13.7750,S
1303,1,1,female,37.0,1,0,90.0000,Q
1304,1,3,female,28.0,0,0,7.7750,S


In [ ]:
#문자 데이터 처리
DF_Hand1["Sex"] = np.where(DF_Hand1["Sex"].to_numpy() == "male", 0, 1)
DF_Hand1["Embarked"] = np.where(DF_Hand1["Embarked"].to_numpy() == "C", 0, np.where(DF_Hand1["Embarked"].to_numpy() == "Q", 1, 2))
DF_Hand1

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,0,22.0,1,0,7.2500,2
2,1,1,1,38.0,1,0,71.2833,0
3,1,3,1,26.0,0,0,7.9250,2
4,1,1,1,35.0,1,0,53.1000,2
5,0,3,0,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...,...
1301,1,3,1,3.0,1,1,13.7750,2
1303,1,1,1,37.0,1,0,90.0000,1
1304,1,3,1,28.0,0,0,7.7750,2


데이터셋 분리 및 표준화 
-------------------------

In [ ]:
# Label 생성
y_test, y_train = DF_Hand1["Survived"][:300].to_numpy(), DF_Hand1["Survived"][300:].to_numpy()

# Dataset 생성
del(DF_Hand1["Survived"])
X_test, X_train = DF_Hand1[:300].values, DF_Hand1[300:].values

In [ ]:
X_train

array([[  1.    ,   0.    ,  27.    , ...,   2.    , 211.5   ,   0.    ],
       [  3.    ,   0.    ,  20.    , ...,   0.    ,   4.0125,   0.    ],
       [  3.    ,   0.    ,  19.    , ...,   0.    ,   7.775 ,   2.    ],
       ...,
       [  3.    ,   1.    ,  28.    , ...,   0.    ,   7.775 ,   2.    ],
       [  1.    ,   1.    ,  39.    , ...,   0.    , 108.9   ,   0.    ],
       [  3.    ,   0.    ,  38.5   , ...,   0.    ,   7.25  ,   2.    ]])

In [ ]:
y_test

array([0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1])

In [ ]:
# 표준화
age_min = np.min(X_test[:, 2])
age_max = np.max(X_test[:, 2])

Fare_min = np.min(X_test[:, 5])
Fare_max = np.max(X_test[:, 5])

X_train[:, 2] = (X_train[:, 2] - age_min)/(age_max - age_min)
X_test[:, 2] = (X_test[:, 2] - age_min)/(age_max - age_min)

X_train[:, 5] = (X_train[:, 5 ] - Fare_min)/(Fare_max - Fare_min)
X_test[:, 5] = (X_test[:, 5] - Fare_min)/(Fare_max - Fare_min)

In [ ]:
X_train

array([[1.        , 0.        , 0.3729514 , ..., 2.        , 0.41282051,
        0.        ],
       [3.        , 0.        , 0.27319367, ..., 0.        , 0.00783188,
        0.        ],
       [3.        , 0.        , 0.25894257, ..., 0.        , 0.01517579,
        2.        ],
       ...,
       [3.        , 1.        , 0.38720251, ..., 0.        , 0.01517579,
        2.        ],
       [1.        , 1.        , 0.54396466, ..., 0.        , 0.21255864,
        0.        ],
       [3.        , 0.        , 0.53683911, ..., 0.        , 0.01415106,
        2.        ]])

In [ ]:
X_test

array([[3.        , 0.        , 0.30169588, ..., 0.        , 0.01415106,
        2.        ],
       [1.        , 1.        , 0.52971355, ..., 0.        , 0.13913574,
        0.        ],
       [3.        , 1.        , 0.3587003 , ..., 0.        , 0.01546857,
        2.        ],
       ...,
       [1.        , 0.        , 0.30169588, ..., 0.        , 0.26473857,
        0.        ],
       [3.        , 1.        , 0.0309249 , ..., 1.        , 0.04113566,
        2.        ],
       [3.        , 1.        , 0.30169588, ..., 0.        , 0.01415106,
        2.        ]])

In [ ]:
# 모델 생성
model = keras.Sequential()
model.add(Dense(128, activation = "relu"))
model.add(Dense(64, activation = "relu"))
model.add(Dense(32, activation = "relu"))
model.add(Dense(16, activation = "relu"))
model.add(Dense(1, activation = "sigmoid"))

In [ ]:
# 모델 Compile
opt = keras.optimizers.Adam(learning_rate = 0.005)
model.compile(optimizer = opt,
              loss = "binary_crossentropy",
              metrics = ["binary_accuracy"])

In [ ]:
model.fit(X_train, y_train, epochs = 500)

Epoch 1/500
24/24 [==============================] - 2s 7ms/step - loss: 0.4530 - binary_accuracy: 0.8183
Epoch 2/500
24/24 [==============================] - 0s 4ms/step - loss: 0.3653 - binary_accuracy: 0.8816
Epoch 3/500
24/24 [==============================] - 0s 5ms/step - loss: 0.3465 - binary_accuracy: 0.8735
Epoch 4/500
24/24 [==============================] - 0s 7ms/step - loss: 0.3298 - binary_accuracy: 0.8802
Epoch 5/500
24/24 [==============================] - 0s 5ms/step - loss: 0.3318 - binary_accuracy: 0.8816
Epoch 6/500
24/24 [==============================] - 0s 4ms/step - loss: 0.3381 - binary_accuracy: 0.8721
Epoch 7/500
24/24 [==============================] - 0s 4ms/step - loss: 0.3507 - binary_accuracy: 0.8681
Epoch 8/500
24/24 [==============================] - 0s 6ms/step - loss: 0.3311 - binary_accuracy: 0.8775
Epoch 9/500
24/24 [==============================] - 0s 4ms/step - loss: 0.3208 - binary_accuracy: 0.8829
Epoch 10/500
24/24 [==========================

In [ ]:
pred = model.predict(X_test).reshape(X_test.shape[0])
pred = np.where(pred > 0.5, 1, 0)
accuracy = 1 - (np.where((pred - y_test) == 0, 0, 1).sum()/len(y_test))
print("Accuracy: ", accuracy)

Accuracy:  0.78
